In [1]:
import pandas as pd
import os


In [2]:
pd.set_option("display.max_rows", None, "display.max_columns",
              None, 'display.max_colwidth', None)

In [3]:
path_dataset = os.path.join(os.path.dirname(os.getcwd()), 'Dataset')


In [19]:
# combine issues and questions

df_issues = pd.read_json(os.path.join(path_dataset, 'issues_original.json'))
df_questions = pd.read_json(os.path.join(
    path_dataset, 'questions_original.json'))

df_issues['Challenge_link'] = df_issues['Issue_link']
df_issues['Challenge_original_content'] = df_issues['Issue_original_content']
df_issues['Challenge_original_content_gpt_summary'] = df_issues['Issue_original_content_gpt_summary']
df_issues['Challenge_preprocessed_content'] = df_issues['Issue_preprocessed_content']
df_issues['Challenge_creation_time'] = df_issues['Issue_creation_time']
df_issues['Challenge_comment_count'] = df_issues['Issue_comment_count']
df_issues['Challenge_score'] = df_issues['Issue_upvote_count'] - df_issues['Issue_downvote_count']

df_questions['Challenge_link'] = df_questions['Question_link']
df_questions['Challenge_original_content'] = df_questions['Question_original_content']
df_questions['Challenge_original_content_gpt_summary'] = df_questions['Question_original_content_gpt_summary']
df_questions['Challenge_preprocessed_content'] = df_questions['Question_preprocessed_content']
df_questions['Challenge_creation_time'] = df_questions['Question_creation_time']
df_questions['Challenge_comment_count'] = df_questions['Question_comment_count']
df_questions['Challenge_score'] = df_questions['Question_score']

df_questions['Solution_original_content'] = df_questions['Answer_original_content']
df_questions['Solution_original_content_gpt_summary'] = df_questions['Answer_original_content_gpt_summary']
df_questions['Solution_preprocessed_content'] = df_questions['Answer_preprocessed_content']

del df_issues['Issue_title']
del df_issues['Issue_body']
del df_issues['Issue_link']
del df_issues['Issue_creation_time']
del df_issues['Issue_comment_count']
del df_issues['Issue_upvote_count']
del df_issues['Issue_downvote_count']
del df_issues['Issue_original_content']
del df_issues['Issue_original_content_gpt_summary']
del df_issues['Issue_preprocessed_content']

del df_questions['Question_title']
del df_questions['Question_body']
del df_questions['Question_link']
del df_questions['Question_creation_time']
del df_questions['Question_comment_count']
del df_questions['Question_score']
del df_questions['Question_original_content']
del df_questions['Question_original_content_gpt_summary']
del df_questions['Question_preprocessed_content']

del df_questions['Answer_original_content']
del df_questions['Answer_original_content_gpt_summary']
del df_questions['Answer_preprocessed_content']

df_all = pd.concat([df_issues, df_questions], ignore_index=True)
df_all.to_json(os.path.join(path_dataset, 'all_original.json'),
               indent=4, orient='records')

In [55]:
# preprocess challenges and solutions

from gensim.parsing.preprocessing import STOPWORDS, remove_stopwords, preprocess_string

stop_words_challenge = ['user', 'encount', 'attempt', 'unabl', 'try', 'seek', 'look', 'face', 'experi', 'struggl', 'us', 'challeng', 'difficulti', 'issu', 'error', 'problem', 'question']
stop_words_solution = ['user', 'provid', 'need', 'includ', 'possibl', 'respond', 'suggest', 'resolv', 'differ', 'solut', 'answer', 'challeng', 'difficulti', 'issu', 'error', 'problem', 'question']

stop_words_challenge = STOPWORDS.union(stop_words_challenge)
stop_words_solution = STOPWORDS.union(stop_words_solution)

df_all = pd.read_json(os.path.join(path_dataset, 'all_original.json'))

for index, row in df_all.iterrows():
    df_all.at[index, 'Challenge_original_content'] = remove_stopwords(' '.join(preprocess_string(row['Challenge_original_content'].replace('Title: ', '').replace('; Content:', ''))), stopwords=stop_words_challenge)
    df_all.at[index, 'Challenge_preprocessed_content'] = remove_stopwords(' '.join(preprocess_string(row['Challenge_preprocessed_content'].replace('Title: ', '').replace('; Content:', ''))), stopwords=stop_words_challenge)
    df_all.at[index, 'Challenge_original_content_gpt_summary'] = remove_stopwords(' '.join(preprocess_string(row['Challenge_original_content_gpt_summary'])), stopwords=stop_words_challenge)
    
    if row['Solution_original_content']:
        df_all.at[index, 'Solution_original_content'] = remove_stopwords(' '.join(preprocess_string(row['Solution_original_content'].replace('Title: ', '').replace('; Content:', ''))), stopwords=stop_words_solution)
        df_all.at[index, 'Solution_preprocessed_content'] = remove_stopwords(' '.join(preprocess_string(row['Solution_preprocessed_content'].replace('Title: ', '').replace('; Content:', ''))), stopwords=stop_words_solution)
        df_all.at[index, 'Solution_original_content_gpt_summary'] = remove_stopwords(' '.join(preprocess_string(row['Solution_original_content_gpt_summary'])), stopwords=stop_words_solution)
    
df_all.to_json(os.path.join(path_dataset, 'all_preprocessed.json'),
               indent=4, orient='records')

In [56]:
df_all = pd.read_json(os.path.join(path_dataset, 'all_preprocessed.json'))

# remove issues with uninformed content
for index, row in df_all.iterrows():
    if len(row['Challenge_original_content'].split()) < 6 or len(row['Challenge_original_content']) < 30:
        print(row['Challenge_original_content'])
        df_all.drop(index, inplace=True)
    elif row['Solution_original_content'] and (len(row['Solution_original_content'].split()) < 6 or len(row['Solution_original_content']) < 30):
        print(row['Solution_original_content'])
        df_all.drop(index, inplace=True)

df_all.to_json(os.path.join(path_dataset, 'all_filtered.json'),
               indent=4, orient='records')

modulenotfounderror modul tensorboard
combin param param work
load
deploy fail
log val loss
fix import
tensorboard default logger option project
logger
let know happen
file pin azurestor
leav case come accross
forc gpu help devic devic gpu
tri work let file transient
aw cli aw sdk exampl
accord document maximum
add run flush end script
csv file folder recordio
begin list
convert dataset modul
cloudform templat fix sure
sure estim framework version version
googl code work
littl involv ye
version correct
situat chang git avail
local comput pipelin articl
think chang log stream servic
current python confid interv
fix updat core
current bring data server
support hive adl
fit pass paramet job yourjobnam
like templat run creat instanc
bug got correct todai close
pipelin moment
vpc endpoint api api st
delet creat endpoint fix microsoft
file path ensur correct path
upgrad sqlalchemi solv
fix version sdk
close solv wrong
current metric publish
network set
chang bucket file work
tri paramet regi

In [ ]:
df_all = pd.read_json(os.path.join(path_dataset, 'all_topics.json'))
df_all['Challenge_creation_date'] = ''
# dates = []

for index, row in df_all.iterrows():
    if 'Git' in row['Platform']:
        date = pd.to_datetime(row['Issue_creation_time'])
    else:
        date = pd.to_datetime(row['Question_creation_time'])
    df_all.at[index, 'Challenge_creation_date'] = date.date()
    # dates.append(date.date())

# min(dates), max(dates)

In [ ]:
df_challenge = df_all.groupby(
    'Challenge_topic').size().reset_index(name='Count')
df_challenge
